# GIS module

Les opérations GIS font partie intégrantes des opérations réalisées en hydrologie. Cette page 

In [1]:
import geopandas as gpd
import pandas as pd
import warnings
import leafmap

from pathlib import Path

import xhydro.gis as xhgis

## Creating a map

In [2]:
m = leafmap.Map(center=(48.63, -74.71), 
                zoom=3,
                basemap="USGS Hydrography")
m

Map(center=[48.63, -74.71], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

## Watershed delineation

### a) From a list of coordinates

In [3]:
lng_lat = [(-69.81971, 48.14720), # Lac Saint-Jean watershed
           (-74.393438, 45.572442) # Ottawa river watershed
          ]

### b) From markers on a map

![test](../../docs/_static/_images/example_draw_marker,png)

In [4]:
# The following code is only useful for the documentation. You should instead remove this code and
# interact with the map as shown above by positionning markers at sites of interest
m.draw_features = [{'type': 'Feature',
  'properties': {},
  'geometry': {'type': 'Point', 'coordinates': [-73.118597, 46.042467]}}] # Richelieu watershed

In [5]:
%%time

gdf = xhgis.watershed_delineation(coordinates=lng_lat,
                                  map=m)
gdf

CPU times: user 4.02 s, sys: 581 ms, total: 4.6 s
Wall time: 4.54 s


,HYBAS_ID,Upstream Area (sq. km).,geometry,category,color
0,7120034330,87595.8,"POLYGON ((-74.37864 48.88141, -74.37452 48.886...",3,#41b6c4
1,7120398781,144026.8,"POLYGON ((-80.07991 46.77860, -80.08529 46.782...",5,#081d58
2,7120382860,23717.7,"POLYGON ((-73.77437 43.36757, -73.77557 43.388...",1,#ffffd9


In [6]:
m.zoom_to_gdf(gdf)

### c) From [xdatasets](https://github.com/hydrologie/xdatasets) (Not implemented yet)

This functionality fetches a list of basins from [xdatasets](https://github.com/hydrologie/xdatasets)' supported datasets, and upon request, [xdatasets](https://github.com/hydrologie/xdatasets) provides a `gpd.GeoDataFrame` containing the precalculated boundaries for these basins.

## Extract geographical watershed properties

a) Let's first 

In [7]:
xhgis.watershed_properties(gdf)

,HYBAS_ID,Upstream Area (sq. km).,category,color,area,perimeter,gravelius,centroid
0,7120034330,87595.8,3,#41b6c4,8.673176e+10,2.576545e+06,2.467991,"(-71.89076256059744, 49.662875012320185)"
1,7120398781,144026.8,5,#081d58,1.440397e+11,3.172740e+06,2.358242,"(-77.22270959350962, 46.711099829791415)"
2,7120382860,23717.7,1,#ffffd9,2.393397e+10,1.336228e+06,2.436509,"(-73.18670092650645, 44.468227773777706)"


In [8]:
xhgis.watershed_properties(gdf[['HYBAS_ID', 'geometry']],
                           unique_id='HYBAS_ID',
                           output_format='xarray')

<xarray.Dataset>
Dimensions:    (HYBAS_ID: 3)
Coordinates:
  * HYBAS_ID   (HYBAS_ID) int64 7120034330 7120398781 7120382860
Data variables:
    area       (HYBAS_ID) float64 8.673e+10 1.44e+11 2.393e+10
    perimeter  (HYBAS_ID) float64 2.577e+06 3.173e+06 1.336e+06
    gravelius  (HYBAS_ID) float64 2.468 2.358 2.437
    centroid   (HYBAS_ID) object (-71.89076256059744, 49.662875012320185) ......